# IMPORTING REQUIRED LIBRARIES

In [3]:
from selenium import webdriver                          #For opening browser       
import pandas as pd                                     #For creating dataframe
from bs4 import BeautifulSoup                           #For scraping data
import time                                             #For waiting
import numpy as np                                      #For array
from tqdm import tqdm                                   #For progress bar

# OPENING REQUIRED SITE IN BROWSER

In [4]:
driver = webdriver.Firefox()

In [5]:
driver.get('https://m.imdb.com/chart/starmeter/')

# GETTING HTML CODE OF OPENED WEBPAGE

In [6]:
html = driver.page_source

# CONVERTING EXTRACTED HTML CODE INTO SOUP TO SCRAP DATA

In [7]:
soup = BeautifulSoup(html,"html.parser")

# EXTRACTING NAMES OF CELEBRITIES AND LINKS TO THEIR INDIVIDUAL WEBPAGES

In [9]:
a_list = soup.find_all("a")
name_list = []
link_list = []
for i in range(45,345,3):
    name_list.append(a_list[i].get('aria-label'))
    link_list.append(a_list[i].get('href'))

# EXTRACTING THE FAMOUS WORK OF THESE CELEBRITIES

In [11]:
famous_list = []
a_list1 = soup.findAll('a',{'data-testid':'nlib-known-for-title'})
for i in a_list1:
    famous_list.append(i.text)

# FUNCTION TO EXTRACT DATE OF BIRTH OF THE CELEBRITIES

In [12]:
def get_dob(soup):
    try:
        sec = soup.findAll('section', {'data-testid':'PersonalDetails'})[0]
        dob = sec.findAll('li',{'data-testid':'nm_pd_bl'})[0].findAll('li')[0].text
    except Exception as e:
        dob = np.nan
    return dob

# FUNCTION TO EXTRACT HEIGHT OF THE CELEBRITIES

In [13]:
def get_height(soup):
    try:
        sec = soup.findAll('section', {'data-testid':'PersonalDetails'})[0]
        height = sec.findAll('li',{'data-testid':'nm_pd_he'})[0].findAll('li')[0].text
    except Exception as e:
        height = np.nan
    return height

# FUNCTION TO EXTRACT BIRTHPLACE OF THE CELEBRITIES

In [14]:
def get_birthplace(soup):
    try:
        sec = soup.findAll('section', {'data-testid':'PersonalDetails'})[0]
        bp = sec.findAll('li',{'data-testid':'nm_pd_bl'})[0].findAll('li')[1].text
    except Exception as e:
        bp = np.nan
    return bp

# FUNCTION TO EXTRACT AWARD DATA OF THE CELEBRITIES

In [15]:
def get_awards(soup):
    try:
        div_list = soup.findAll('div', {'data-testid':'awards'})
        awards = div_list[0].findAll('li',{'class':'ipc-inline-list__item'})[0].text
    except Exception as e:
        awards = np.nan
    return awards

# FUNCTION TO EXTRACT ROLES OF THE CELEBRITIES

In [16]:
def get_role(soup):
    role_list = []
    try:
        div_list = soup.findAll('div', {'class':'sc-7c4234bd-0 jwsqEY'})[0].findAll('li')
        for i in div_list:
            role_list.append(i.text)
    except Exception as e:
        role_list.append(np.nan)
    return role_list

# MAIN LOOP TO ITERATE THROUGH EACH CELEBRITY'S WEBPAGE AND EXTRACT DATA

In [17]:
dob_list = []
height_list = []
role_list = []
award_list = []
birth_list = []
for i in tqdm(link_list, desc="Processing Links"):
    driver.get(f'https://m.imdb.com/{i}')
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html,"html.parser")
    dob_list.append(get_dob(soup))
    height_list.append(get_height(soup))
    birth_list.append(get_birthplace(soup))
    role_list.append(get_role(soup))
    award_list.append(get_awards(soup))

Processing Links: 100%|██████████| 100/100 [13:25<00:00,  8.05s/it]


# CREATING DATAFRAME FOR COLLECTED DATASET 

In [18]:
df = pd.DataFrame({
    'Rank':np.arange(1,101),
    'Name': name_list,
    'Date_of_birth': dob_list,
    'height': height_list,
    'Role': role_list,
    'Awards': award_list,
    'Famous_for': famous_list,
    'Birth_place': birth_list
})

# CHECKING THE DATASET

In [19]:
df.head()

,Rank,Name,Date_of_birth,height,Role,Awards,Famous_for,Birth_place
0,1,Carla Gugino,"August 29, 1971",5′ 5″ (1.65 m),"[Actress, Producer, Soundtrack]",3 wins & 15 nominations,Watchmen (2009),"Sarasota, Florida, USA"
1,2,Lily Gladstone,"August 2, 1986",NaN,"[Actress, Casting Department, Writer]",8 wins & 21 nominations,Certain Women (2016),"Browning, Montana"
2,3,Shira Haas,"May 11, 1995",NaN,"[Actress, Soundtrack]",7 wins & 15 nominations total,Shtisel (2013–2021),"Tel Aviv, Israel"
3,4,Kate Siegel,"August 9, 1982",5′ 8″ (1.73 m),"[Actress, Writer, Make-Up Department]",8 nominations,Hush (2016),"Silver Spring, Maryland, USA"
4,5,Willa Fitzgerald,"January 17, 1991",5′ 5″ (1.65 m),"[Actress, Soundtrack]",3 nominations,Scream: The TV Series (2015–2016),"Nashville, Tennessee, USA"


In [20]:
df.tail()

,Rank,Name,Date_of_birth,height,Role,Awards,Famous_for,Birth_place
95,96,Henry Cavill,"May 5, 1983",6′ 1″ (1.85 m),"[Actor, Additional Crew, Producer]",3 wins & 15 nominations,Man of Steel (2013),"Jersey, Channel Islands"
96,97,Juno Temple,"July 21, 1989",5′ 2″ (1.57 m),"[Actress, Soundtrack]",6 wins & 19 nominations total,Atonement (2007),"London, England, UK"
97,98,Miles McKenna,"November 2, 1995",5′ 7½″ (1.71 m),"[Actor, Producer]",1 win,Nocturne (2020),NaN
98,99,Pom Klementieff,"May 3, 1986",5′ 7″ (1.70 m),"[Actress, Writer]",1 win,Guardians of the Galaxy Vol. 2 (2017),"Quebec City, Quebec, Canada"
99,100,David Fincher,"August 28, 1962",6′ 0½″ (1.84 m),"[Producer, Director, Writer]",72 wins & 150 nominations total,Se7en (1995),"Denver, Colorado, USA"


In [21]:
df.sample(20)

,Rank,Name,Date_of_birth,height,Role,Awards,Famous_for,Birth_place
49,50,Michael Trucco,June 22,6′ 3″ (1.91 m),"[Actor, Director]",1 nomination,Next (2007),"San Mateo, California, USA"
40,41,Vinessa Shaw,"July 19, 1976",5′ 8″ (1.73 m),"[Actress, Director, Producer]",9 nominations,Hocus Pocus (1993),"Los Angeles, California, USA"
72,73,Jessica Barden,"July 21, 1992",5′ 2″ (1.57 m),[Actress],1 win & 3 nominations,Hanna (2011),"Northallerton, North Yorkshire, England, UK"
36,37,Zach Gilford,"January 14, 1982",5′ 8″ (1.73 m),"[Actor, Producer, Soundtrack]",2 wins & 10 nominations,The Purge: Anarchy (2014),"Evanston, Illinois, USA"
96,97,Juno Temple,"July 21, 1989",5′ 2″ (1.57 m),"[Actress, Soundtrack]",6 wins & 19 nominations total,Atonement (2007),"London, England, UK"
74,75,Ana de Armas,"April 30, 1988",5′ 6¼″ (1.68 m),"[Actress, Producer]",11 wins & 34 nominations total,Blade Runner 2049 (2017),"Havana, Cuba"
67,68,Florence Pugh,"January 3, 1996",5′ 3¾″ (1.62 m),"[Actress, Producer, Soundtrack]",39 wins & 95 nominations total,The Falling (2014),"Oxford, Oxfordshire, England, UK"
75,76,Ryan Reynolds,"October 23, 1976",6′ 2″ (1.88 m),"[Actor, Producer, Writer]",21 wins & 63 nominations total,Deadpool 2 (2018),"Vancouver, British Columbia, Canada"
77,78,George Parker,NaN,NaN,[Actor],NaN,Bodies (2023– ),NaN
14,15,Brie Larson,"October 1, 1989",5′ 6¾″ (1.70 m),"[Actress, Producer, Director]",66 wins & 68 nominations total,Room (2015),"Sacramento, California, USA"


# SAVING THE DATASET

In [22]:
df.to_csv('celebrity_data.csv',index= False)